<a href="https://colab.research.google.com/github/MC855FilmesParaTi/machinelearning/blob/FPTI-21/notebooks/3-lfb-split-data_ratings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd drive/Shareddrives/Filmes Pra TI - Machine Learning/Clone do Projeto - Luana/machinelearning

/content/drive/Shareddrives/Filmes Pra TI - Machine Learning/Clone do Projeto - Luana/machinelearning


In [3]:
import pandas as pd
import numpy as np
import time
import datetime

all paths are related to drive folder

In [4]:
data_dir = 'data/processed/'

In [8]:
rating_df = pd.read_csv(data_dir + 'rating.csv')
rating_df.head()

,userId,movieId,rating,timestamp,Liked,timestamp2
0,28507,1176,4.0,1995-01-09 11:46:44,1,789652004
1,131160,1079,3.0,1995-01-09 11:46:49,1,789652009
2,131160,47,5.0,1995-01-09 11:46:49,1,789652009
3,131160,21,3.0,1995-01-09 11:46:49,1,789652009
4,85252,45,3.0,1996-01-29 00:00:00,1,822873600


In [21]:
rating_df.movieId.value_counts().index

Int64Index([   296,    356,    318,    593,    480,    260,    110,    589,
              2571,    527,
            ...
            109810, 109790, 109643, 109798,  27395,  42407, 109779, 109776,
            109771, 131262],
           dtype='int64', length=26744)

In [22]:
rating_df.movieId.value_counts().values

array([67310, 66172, 63366, ...,     1,     1,     1])

In [23]:
# getting most rated movies
N = 10000
most_rated_movies = rating_df.movieId.value_counts().index[:N]
most_rated_movies

Int64Index([  296,   356,   318,   593,   480,   260,   110,   589,  2571,
              527,
            ...
            57949, 60137,  6050,   402, 26043, 53143,  3878,  6588,   871,
             7109],
           dtype='int64', length=10000)

In [30]:
# getting users that have made more ratings so far
M = 10000
most_raters_users = rating_df.userId.value_counts().index[:M]

In [33]:
# filtering ratings to both use these movies and users only
reduced_ratings = rating_df[rating_df.userId.isin(most_raters_users)]
reduced_ratings = reduced_ratings[reduced_ratings.movieId.isin(most_rated_movies)]
reduced_ratings.shape

(7579274, 6)

Reduced data to 20M to 7M

In [38]:
reduced_ratings.sort_values(by='timestamp2', inplace=True)

In [39]:
reduced_ratings.to_csv(data_dir + 'reduced_ratings.csv', index=False)

Split data into train, val and test sets

In [40]:
users = reduced_ratings.userId.unique()
users.sort()
users

array([    11,     24,     54, ..., 138411, 138437, 138474])

In [36]:
from sklearn.model_selection import train_test_split

In [41]:
for i, user in enumerate(users):
    # ratings of one user sorted by timestamp2
    group = reduced_ratings[reduced_ratings.userId == user]
    train, test = train_test_split(group, test_size=0.3, shuffle=False)
    val, test = train_test_split(test, test_size=0.5, shuffle=False)

    if i == 0:
        test.to_csv(data_dir + 'reduced_test_rating.csv', index=False)
        train.to_csv(data_dir + 'reduced_train_rating.csv', index=False)
        val.to_csv(data_dir + 'reduced_val_rating.csv', index=False)
    else:
        # append to existing csv file
        test.to_csv(data_dir + 'reduced_test_rating.csv', index=False, header=False, mode='a')
        train.to_csv(data_dir + 'reduced_train_rating.csv', index=False, header=False, mode='a')
        val.to_csv(data_dir + 'reduced_val_rating.csv',  index=False, header=False, mode='a')

    if (i % 1000) == 0:
        print(user)

11
13174
26651
41127
55122
69481
83081
96859
110392
124123


In [42]:
train = pd.read_csv(data_dir + 'reduced_train_rating.csv')
val = pd.read_csv(data_dir + 'reduced_val_rating.csv')
test = pd.read_csv(data_dir + 'reduced_test_rating.csv')

In [43]:
train.shape

(5301046, 6)

In [44]:
val.shape

(1136607, 6)

In [45]:
test.shape

(1141621, 6)

In [46]:
train.drop_duplicates(inplace=True)
val.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)